In [4]:
import networkx as nx
import pandas as pd
import numpy as np
from PyKakao import Local
import json
import requests

api = Local(service_key = "nopenotachance")

def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + "isaidnope"}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['x']), float(match_first['y'])

#coordinates are DD (decimal degrees)* 

In [10]:
cafefilter = pd.read_csv("Cheonan_Cafe_df.csv", encoding = "utf-8")

In [12]:
cafefilter.head()

,Title,Address
0,카페이숲,충남 천안시 서북구 성거읍 남창마을1길 32 1층
1,블렌데렌,"충남 천안시 동남구 먹거리8길 5 1, 2층"
2,그리고스페셜티,충남 천안시 동남구 유량로 104
3,오월의숲,충남 천안시 서북구 세관길 78
4,유얼스,충남 천안시 서북구 불당31길 37


In [18]:
def add_lat_lon_columns(df):
    longitudes = [] #경도
    latitudes = [] #위도
    for address in df['Address']:
        try:
            lon,lat = addr_to_lat_lon(address)
            longitudes.append(lon)
            latitudes.append(lat)
        except Exception as e:
            print(f"Error processing address '{address}': {e}")
            latitudes.append(None)
            longitudes.append(None)
    df['Longitude'] = longitudes
    df['Latitude'] = latitudes
    return df

In [20]:
cafefilter2 = add_lat_lon_columns(cafefilter)

Error processing address '충남 천안시 동남구 먹거리8길 5 1, 2층': list index out of range
Error processing address '충남 천안시 동남구 목천읍 동리2길 183 1~4층': list index out of range
Error processing address '충남 천안시 동남구 통세골2길 41 1,2층': list index out of range
Error processing address '충남 천안시 서북구 업성수변로 120 1,2층': list index out of range
Error processing address '충남 천안시 동남구 목천읍 서리1길 26 1,2층': list index out of range
Error processing address '충남 천안시 서북구 성환읍 왕지1길 121 1,2층': list index out of range
Error processing address '충남 천안시 동남구 먹거리10길 6 1,2층': list index out of range


In [24]:
cafefilter2

,Title,Address,Longitude,Latitude
0,카페이숲,충남 천안시 서북구 성거읍 남창마을1길 32 1층,127.195422,36.862185
1,블렌데렌,"충남 천안시 동남구 먹거리8길 5 1, 2층",NaN,NaN
2,그리고스페셜티,충남 천안시 동남구 유량로 104,127.172754,36.807128
3,오월의숲,충남 천안시 서북구 세관길 78,127.145208,36.823195
4,유얼스,충남 천안시 서북구 불당31길 37,127.105355,36.810215
...,...,...,...,...
276,하임,충남 천안시 서북구 늘푸른3길 41 인트로타운 1층 108호,127.131493,36.826868
277,커방,충남 천안시 동남구 중앙로 30 2동 1층 2호,127.153430,36.801125
278,카페티핫,충남 천안시 서북구 원두정3길 27 2층 201호,127.137161,36.832650
279,카페다빈제과,충남 천안시 서북구 한들3로 35-11 1층,127.122670,36.826581


In [26]:
def get_address(address):
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address}'
    name = ''
    headers = {"Authorization": "KakaoAK " + ""}
    try:
        name = requests.get(url, headers=headers).json()['documents'][0]['address']['region_3depth_h_name']
    except Exception as e:
        print(e)
        print(address)
    return name

get_address(cafefilter2["Address"][0])

'성거읍'

In [30]:
def add_dong_name(df):
    dong_name = []
    for address in df['Address']:
        try:
            dong = get_address(address)
            dong_name.append(dong)
        except Exception as e:
            print(f"Error processing address '{address}': {e}")
            dong_name.append(None)
    df['법정동'] = dong_name
    return df

cafefilter2_dong = add_dong_name(cafefilter2)
cafefilter2_dong

list index out of range
충남 천안시 동남구 먹거리8길 5 1, 2층
list index out of range
충남 천안시 동남구 목천읍 동리2길 183 1~4층
list index out of range
충남 천안시 동남구 통세골2길 41 1,2층
list index out of range
충남 천안시 서북구 업성수변로 120 1,2층
list index out of range
충남 천안시 동남구 목천읍 서리1길 26 1,2층
list index out of range
충남 천안시 서북구 성환읍 왕지1길 121 1,2층
list index out of range
충남 천안시 동남구 먹거리10길 6 1,2층


,Title,Address,Longitude,Latitude,법정동
0,카페이숲,충남 천안시 서북구 성거읍 남창마을1길 32 1층,127.195422,36.862185,성거읍
1,블렌데렌,"충남 천안시 동남구 먹거리8길 5 1, 2층",NaN,NaN,
2,그리고스페셜티,충남 천안시 동남구 유량로 104,127.172754,36.807128,원성1동
3,오월의숲,충남 천안시 서북구 세관길 78,127.145208,36.823195,성정2동
4,유얼스,충남 천안시 서북구 불당31길 37,127.105355,36.810215,불당2동
...,...,...,...,...,...
276,하임,충남 천안시 서북구 늘푸른3길 41 인트로타운 1층 108호,127.131493,36.826868,부성2동
277,커방,충남 천안시 동남구 중앙로 30 2동 1층 2호,127.153430,36.801125,중앙동
278,카페티핫,충남 천안시 서북구 원두정3길 27 2층 201호,127.137161,36.832650,부성2동
279,카페다빈제과,충남 천안시 서북구 한들3로 35-11 1층,127.122670,36.826581,백석동


In [33]:
cafefilter2_dong.head(10)

,Title,Address,Longitude,Latitude,법정동
0,카페이숲,충남 천안시 서북구 성거읍 남창마을1길 32 1층,127.195422,36.862185,성거읍
1,블렌데렌,"충남 천안시 동남구 먹거리8길 5 1, 2층",NaN,NaN,
2,그리고스페셜티,충남 천안시 동남구 유량로 104,127.172754,36.807128,원성1동
3,오월의숲,충남 천안시 서북구 세관길 78,127.145208,36.823195,성정2동
4,유얼스,충남 천안시 서북구 불당31길 37,127.105355,36.810215,불당2동
5,풍세커피,충남 천안시 동남구 풍세면 한우물2길 90 2동 1층,127.106483,36.740875,풍세면
6,교토리,충남 천안시 동남구 북면 위례성로 782,127.271452,36.831803,북면
7,피노카페,충남 천안시 동남구 풍세면 휴양림길 99-23,127.092565,36.729021,풍세면
8,마리스,충남 천안시 동남구 천호지길 11,127.172957,36.834352,신안동
9,마젠타,충남 천안시 서북구 성성2길 81 1동 2~4층,127.129597,36.846493,부성2동


In [51]:
cafefilter2_dong.to_csv('out2.csv', encoding='utf-8')